In [1]:
!python -V

Python 3.9.23


In [2]:
import pandas as pd


In [4]:
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

2026/01/29 17:46:33 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp-exercises/mlartifacts/1', creation_time=1769708793769, experiment_id='1', last_update_time=1769708793769, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
import pandas as pd

def read_dataframe(filename):
    df=pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    return df

In [10]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [12]:
len(df_train),len(df_val)

(73908, 61921)

In [15]:
categorical=['PU_DO']  #['PULocationID','DOLocationID']
numerical = ['trip_distance']

dv=DictVectorizer()

train_dicts=df_train[categorical+numerical].to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)

val_dicts=df_val[categorical+numerical].to_dict(orient='records')
X_val=dv.transform(val_dicts)

In [16]:
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

In [17]:
import xgboost as xgb


In [22]:
mlflow.xgboost.autolog(disable=True)


In [23]:
with mlflow.start_run():
    params = {
        'learning_rate': 0.3204931383182283,
        'max_depth': 10,  
        'min_child_weight': 1.0292136879853275,
        'objective': 'reg:squarederror', 
        'reg_alpha': 0.01641795441600401,
        'reg_lambda': 0.018850704093658917,
        'seed': 42
    }
    
    mlflow.log_params(params)
    
    # Εκπαίδευση
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=100, # Βάζουμε 100 για να τελειώσει ΣΦΑΙΡΑ
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
    y_pred=booster.predict(valid)
    rmse=root_mean_squared_error(y_val,y_pred)
    mlflow.log_metric("rmse",rmse)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")

[0]	validation-rmse:9.80062
[1]	validation-rmse:8.40509
[2]	validation-rmse:7.63431
[3]	validation-rmse:7.21840
[4]	validation-rmse:6.98903
[5]	validation-rmse:6.86272
[6]	validation-rmse:6.79360
[7]	validation-rmse:6.75183
[8]	validation-rmse:6.72431
[9]	validation-rmse:6.70801
[10]	validation-rmse:6.69642
[11]	validation-rmse:6.68724
[12]	validation-rmse:6.68005
[13]	validation-rmse:6.67894
[14]	validation-rmse:6.67611
[15]	validation-rmse:6.67264
[16]	validation-rmse:6.66892
[17]	validation-rmse:6.66609
[18]	validation-rmse:6.66227
[19]	validation-rmse:6.65858
[20]	validation-rmse:6.65652
[21]	validation-rmse:6.65351
[22]	validation-rmse:6.65087
[23]	validation-rmse:6.64864
[24]	validation-rmse:6.64681
[25]	validation-rmse:6.64514
[26]	validation-rmse:6.64222
[27]	validation-rmse:6.64150
[28]	validation-rmse:6.63938
[29]	validation-rmse:6.63688
[30]	validation-rmse:6.63500
[31]	validation-rmse:6.63400
[32]	validation-rmse:6.63286
[33]	validation-rmse:6.62999
[34]	validation-rmse:6.6

2026/01/29 18:07:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:07:10] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2026/01/29 18:07:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run adaptable-snake-372 at: http://localhost:5000/#/experiments/1/runs/2db75682d10a48308d4eb3a36159c8b1
🧪 View experiment at: http://localhost:5000/#/experiments/1
